# 0.0 IMPORTS

In [47]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16  # ou outra arquitetura de CNN
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image

from functools import partial


In [48]:
tf.keras.backend.clear_session()

In [49]:
path = 'C:/Users/edils/repos/brain_tumor_detection/data/'

In [61]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rescale=1./255,
    shear_range=0.05,
    brightness_range=[0.1, 1.5],
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    path + 'train/',
    target_size=(150, 150),  # Tamanho das imagens após o redimensionamento
    batch_size=32,
    class_mode='binary'  # Ou 'categorical' se houver mais classes
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(
    path + 'validation/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    path + 'test/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 193 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [62]:
# DefaultConv2D = partial(tf.keras.layers.Conv2D, 
#                         kernel_size=3, 
#                         padding='same', 
#                         activation='relu',
#                         kernel_initializer='he_normal')

# model = tf.keras.models.Sequential([
#     tf.keras.layers.Rescaling(1/255, input_shape=(224,224,3)),
#     DefaultConv2D(filters=64, kernel_size=7),
#     tf.keras.layers.MaxPool2D(),
#     DefaultConv2D(filters=128),
#     DefaultConv2D(filters=128),
#     tf.keras.layers.MaxPool2D(),
#     DefaultConv2D(filters=256),
#     DefaultConv2D(filters=256),
#     tf.keras.layers.MaxPool2D(),
    
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=128,
#                           activation='relu',
#                           kernel_initializer='he_normal'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(units=64,
#                           activation='relu',
#                           kernel_initializer='he_normal'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(units=1, activation='sigmoid')
# ])

def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1/255, input_shape=(150, 150, 3)),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [63]:
model = create_model()

In [64]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [65]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 74, 74, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 41472)            

In [66]:
epochs=30
batch_size=32
early_stopping = EarlyStopping(
                monitor='val_accuracy', 
                mode='max',
                patience=6
)

history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=[early_stopping]
)

Epoch 1/30
7/7 [==============================] - 2s 211ms/step - loss: 0.6917 - accuracy: 0.6114 - val_loss: 0.6857 - val_accuracy: 0.6200
Epoch 2/30
7/7 [==============================] - 1s 187ms/step - loss: 0.6834 - accuracy: 0.6166 - val_loss: 0.6809 - val_accuracy: 0.6200
Epoch 3/30
7/7 [==============================] - 1s 191ms/step - loss: 0.6866 - accuracy: 0.6166 - val_loss: 0.6845 - val_accuracy: 0.6200
Epoch 4/30
7/7 [==============================] - 1s 201ms/step - loss: 0.6810 - accuracy: 0.6166 - val_loss: 0.6709 - val_accuracy: 0.6200
Epoch 5/30
7/7 [==============================] - 1s 200ms/step - loss: 0.6671 - accuracy: 0.6166 - val_loss: 0.6639 - val_accuracy: 0.6200
Epoch 6/30
7/7 [==============================] - 1s 206ms/step - loss: 0.6826 - accuracy: 0.6166 - val_loss: 0.6750 - val_accuracy: 0.6200
Epoch 7/30
7/7 [==============================] - 1s 193ms/step - loss: 0.6622 - accuracy: 0.6166 - val_loss: 0.6610 - val_accuracy: 0.6200


In [67]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

1/1 [==============================] - 0s 40ms/step - loss: 0.7174 - accuracy: 0.5000
Test accuracy: 0.5
